In [ ]:
import pandas as pd # 데이터분석 라이브러리로, 행과 열로 이루어진 데이터 객체를 만들어 다룰 수 있음
import numpy as np #  다차원 배열을 쉽게 처리하고  효율적으로 사용할 수 있도록 하는 라이브러리
import math  # 수학함수 제공
import re  # 정규 표현식 제공
from scipy.sparse import csr_matrix  # (희소행렬) 핸들링 값이 대부분이
import matplotlib.pyplot as plt # 차트(chart)나 플롯(plot)으로 시각화(visulaization)하는 패키지
import seaborn as sns  # Matplotlib을 기반으로 다양한 색상 테마와 통계용 차트 등의 기능을 추가한 시각화 패키지
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate  # evaluate 이부분은 2020년도에 옆과 같이 바꼈다.
sns.set_style("darkgrid")

In [ ]:
!pip install surprise

In [ ]:
from google.colab import drive  # 구글 코랩에서 drive(내 구글 드라이브)를 사용하기 위한 함수
import io
drive.mount('/content/drive')  # 이렇게 하면 내 drive가 설정? 잡힌다.

In [ ]:
filename = '/content/drive/My Drive/data/netfliex_prize_data/combined_data_1.txt' # 경로를 지정해준다.
filename
df1 = pd.read_csv(filename, header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1]) # 내파일을 읽어준다. 그때 0-1열을 가져오고 그 이름을 'Cust_Id', 'Rating'으로 짓는다.

df1['Rating'] = df1['Rating'].astype(float) # rating 칼럼을 float 형식으로 형변환

print('Dataset 1 shape: {}'.format(df1.shape)) # shape(numpy) - 차원을 나타낸다.  즉 df1의 차원을 출력한다.(24058263, 2) 2405만 행에 2열을 의미
print('-Dataset examples-')
print(df1.iloc[::5000000, :])  # pandas - 행 번호를 기준으로 행 데이터 읽기 / [::500000]모든 행을 출력하는데 5000000 간격으로 출력한다./ 열은 모든 열을 출력

In [ ]:
#df2 = pd.read_csv('../input/combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
#df3 = pd.read_csv('../input/combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
#df4 = pd.read_csv('../input/combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])


#df2['Rating'] = df2['Rating'].astype(float)
#df3['Rating'] = df3['Rating'].astype(float)
#df4['Rating'] = df4['Rating'].astype(float)

#print('Dataset 2 shape: {}'.format(df2.shape))
#print('Dataset 3 shape: {}'.format(df3.shape))
#print('Dataset 4 shape: {}'.format(df4.shape))

In [ ]:
# load less data for speed

df = df1  # 위에서 썻던 df1을 df 로 넣는다
#df = df1.append(df2)
#df = df.append(df3)
#df = df.append(df4)

df.index = np.arange(0,len(df)) # df의 길이 만큼을 df의 인덱스로 만든다.
print('Full dataset shape: {}'.format(df.shape)) # df의 차원을 출력한다.
print('-Dataset examples-')
print(df.iloc[::5000000, :])

In [ ]:
#test

p = df.groupby('Rating')['Rating'].agg(['count'])  #('Rating') 이 열을 그룹화하는데 Rating을 집계한다. count 라는 명령어를 통해 개수를 센다.
print(p)

movie_count = df.notnull().sum()[1] # 결측값 여부를 모두 더한 다음 출력24058263
print(movie_count)
cust_count = df['Cust_Id'].nunique() - movie_count # Cust_id 열에서 유니크한 id의 개수(중복을 제거한 사용자) - rating이 없는 사람들 = 고객수()

# get rating count
rating_count = df['Cust_Id'].count() - movie_count # 총 cust_id(총 고객수) - rating이 없는 사람들  = 평점을 준 개수

ax = p.plot(kind = 'barh', legend = False, figsize = (15,10)) # p 에 대한 것을 plot 그리기 

plt.title('Total pool: {:,} Movies, {:,} customers, {:,} ratings given'.format(movie_count, cust_count, rating_count), fontsize=20) # plot의 이름 넣기 


plt.axis('off')


In [ ]:
p = df.groupby('Rating')['Rating'].agg(['count'])  # #('Rating') 이 열을 그룹화하는데 Rating을 집계한다. count 라는 명령어를 통해 개수를 센다.
# 각 raing 별 count한 개수가 나온다. 

# get movie count
movie_count = df.isnull().sum()[1]   # Rating이 없는 사람들 =  결측값 여부를 모두 더한 다음 출력 (출력 4499) ? 이게 rating이 null 인게 moive임 같음

# get customer count
cust_count = df['Cust_Id'].nunique() - movie_count # Cust_id 열에서 유니크한 id의 개수(중복을 제거한 사용자) - rating이 없는 사람들 = 고객수()

# get rating count
rating_count = df['Cust_Id'].count() - movie_count # 총 cust_id(총 고객수) - rating이 없는 사람들  = 평점을 준 개수

ax = p.plot(kind = 'barh', legend = False, figsize = (15,10)) # p 에 대한 것을 plot 그리기 
plt.title('Total pool: {:,} Movies, {:,} customers, {:,} ratings given'.format(movie_count, cust_count, rating_count), fontsize=20) # plot의 이름 넣기 
plt.axis('off')  # 축과 라벨을 끈다

for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i, p.iloc[i-1][0]*100 / p.sum()[0]), color = 'white', weight = 'bold')

# 각각의 rating에 대한 퍼센트를 구해서 출력한다.

In [ ]:
#test

df_nan = pd.DataFrame(pd.isnull(df.Rating)) #df에 있는 rating의 결측치를 제거한다. 그걸 데이터 프레임으로 만든다.
print(df_nan)
df_nan = df_nan[df_nan['Rating'] == True] # 그중 Rating이 ture 인것만 가져온다.
print(df_nan)
df_nan = df_nan.reset_index()  #기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가한다. 
print(df_nan)

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]): #인덱스를 첫번째거는 첫번째 부터 두번째꺼는 0번째 부터 시작해서 두번째꺼는 -1 번째 까지 한다. 
    # numpy approach        # 밑에 두인덱스의 차이를 통해 몰 구하는 지 모르겠다.
    temp = np.full((1,i-j-1), movie_id) # full()은 배열에 사용자가 지정한 값을 넣는데 사용 ((2,3),5) 일경우 2행 3열의 형태에 모든 값이 5이다.
    movie_np = np.append(movie_np, temp)
    movie_id += 1

print(movie_id)


In [ ]:
#test
print(movie_np)
print(df_nan['index'][1:])
print(df_nan['index'][:-1])

In [ ]:
#test
print(df_nan.iloc[-1, 0])
print(len(df))
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
last_record # 1행에서 , df의 길이(24058263) - (24057

In [ ]:
df_nan = pd.DataFrame(pd.isnull(df.Rating)) #df에 있는 rating의 값에 따라 true /false를 나눈다. 그걸 데이터 프레임으로 만든다.
df_nan = df_nan[df_nan['Rating'] == True] # 그중 Rating이 ture 인것만 가져온다.
df_nan = df_nan.reset_index()  #기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가한다. 

movie_np = []
movie_id = 1

# 이부분이 무슨 작업인지 잘 모르겠다.
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):  #인덱스를 첫번째거는 첫번째 부터 두번째꺼는 0번째 부터 시작해서 두번째꺼는 -1 번째 까지 한다.
    # numpy approach                    # 밑에 두인덱스의 차이를 통해 몰 구하는 지 모르겠다.
    temp = np.full((1,i-j-1), movie_id)     # full()은 배열에 사용자가 지정한 값을 넣는데 사용 ((2,3),5) 일경우 2행 3열의 형태에 모든 값이 5이다.
    movie_np = np.append(movie_np, temp) # movie_np에 temp를 덧붙인다.
    movie_id += 1                       # movie_id 를 계속 더해간다.

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id) # 1행에서 , df의 길이(24058263) - (24057834),행열을 만드는데 movie_id는 8997이다.
movie_np = np.append(movie_np, last_record) # movie_np에 last_Record를 덧붙인다. 여기가 새로운 무비 id 가 된다.

print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))

In [ ]:
# remove those Movie ID rows
df = df[pd.notnull(df['Rating'])] # Rating 이 notnull인 행렬을 값들을 가져온다. 

df['Movie_Id'] = movie_np.astype(int)  #타입 형변환 후 movie_id 를 넣는다.
df['Cust_Id'] = df['Cust_Id'].astype(int) # int 형 타입으로 Cust id 를 넣는다.
print('-Dataset examples-')
print(df.iloc[::5000000, :])

In [ ]:
f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)  #('movie_id') 이 열을 그룹화 하고  Rating의 count와 평균을 한다.

df_movie_summary.index = df_movie_summary.index.map(int)  # index를 map을 통해 int형으로 모두 형변환한다 그다음 index를 추가한다.

# 백분위수(Percentile)는 오름차순 정렬했을 때 0을 최소값, 100을 최대값으로 백분율로 나타낸 특정 위치 값입니다.  / round - 반올림한다
#  따라서 사분위수는 25, 50, 75를 기준 점으로 나눠져 1분위부터 4분위까지 존재하게 된다.
# quantile(0.5) 하면 중앙값을 찾을 수 있다.
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)  # 70프로의 분위수를 찾는다.성능측정을 하기 위하여 만든다.
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index # 70프로의 분위수 보다 작은 값들을 drop_movie_list에 넣는다.

print('Movie minimum times of review: {}'.format(movie_benchmark)) #

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f) #('Cust_Id') 이 열을 그룹화 하고  Rating의 count와 mean을 한다.
df_cust_summary.index = df_cust_summary.index.map(int)  # index를 int형으로 형변환한다음에 그 값을 index에 넣어준다. 
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0) # 분위수를 계산해주며, 최소값과 최대값을 등분하여 그 사이를 interpolation 하여 분위수를 계산하는 방식

drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

In [ ]:
# test
print(movie_benchmark)
print(df_movie_summary['count'])
print(round(df_movie_summary['count'].quantile(0.7),0))
a = round(df_movie_summary['count'].quantile(0.5),0)
print(a)
print(df_movie_summary.shape)
print(drop_movie_list.shape)

In [ ]:
print('Original Shape: {}'.format(df.shape))
# isin 구문은 열이 list의 값들을 포함하고 있는 모든 행들을 골라낼 때 주로 쓰인다. 여기서는 df의 값중에 drop_movie_list에 있는 값을 가지고 있는
# 값들을 df 에 넣어준다. 앞에 물결 표시는 not을 뜻한다. 고로 drop_movie_list에 해당 하지 않는 것들을 저장한다. 
print(df)
df = df[~df['Movie_Id'].isin(drop_movie_list)]  
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

In [ ]:
# 피봇테이블(pivot table)이란 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것
df_p = pd.pivot_table(df,values='Rating',index='Cust_Id',columns='Movie_Id') # 들어가는 값은 Rating / index 부분은 Cust_id / columns 는 Movie_id를 넣는다.

print(df_p.shape)
print(df_p)
# Below is another way I used to sparse the dataframe...doesn't seem to work better

#Cust_Id_u = list(sorted(df['Cust_Id'].unique()))
#Movie_Id_u = list(sorted(df['Movie_Id'].unique()))
#data = df['Rating'].tolist()
#row = df['Cust_Id'].astype('category', categories=Cust_Id_u).cat.codes
#col = df['Movie_Id'].astype('category', categories=Movie_Id_u).cat.codes
#sparse_matrix = csr_matrix((data, (row, col)), shape=(len(Cust_Id_u), len(Movie_Id_u)))
#df_p = pd.DataFrame(sparse_matrix.todense(), index=Cust_Id_u, columns=Movie_Id_u)
#df_p = df_p.replace(0, np.NaN)

In [ ]:
title_filename = '/content/drive/My Drive/data/netfliex_prize_data/movie_titles.csv'
df_title = pd.read_csv(title_filename, encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
df_title.set_index('Movie_Id', inplace = True) # index를 movie_id로 쓴다.
print (df_title.head(10))

In [ ]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:100000], reader) # load_from_df - surpise에서 쓰는 함수로서 데이터셋을 가져오는데 사용한다.
# data.split(n_folds=3) 아마도 3개의 폴드로 나누는 거같음 

#정방 행렬이 아닌 행렬 M에 대해서도 고유 분해와 유사한 분해가 가능 이를 특이값 분해(singular value decomposition)이라고 함 svd는 특이값 분해를 해준다.
svd = SVD()

# Run 5-fold cross-validation and print results
#이렇게 하면 3가지 폴드로 나누고 그걸 
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# evaluate(svd, data, measures=['RMSE', 'MAE']) #평가할때 쓰는거 같다.

In [ ]:
df_785314 = df[(df['Cust_Id'] == 785314) & (df['Rating'] == 5)] # 사용자 785314가 평가 점수 5점을 준 movie를 가져온다.
# print(df_785314)
df_785314 = df_785314.set_index('Movie_Id') # 그다음 index를 movie_id로 한다.
# print(df_785314)
df_785314 = df_785314.join(df_title)['Name'] # join 구문으로서  df_title의 name과 연결한다.
print(df_785314) # 출력하면 해당 movie id에 맞는 이름을 가져온다

In [ ]:
user_785314 = df_title.copy() # user에 해당 데이터프레임을 복사한다음에
user_785314 = user_785314.reset_index() # 새로운 index를 생성한다. (1,2,3,4...)
user_785314 = user_785314[~user_785314['Movie_Id'].isin(drop_movie_list)] # 빼야될 movielist 를 제외한 것들을 저장한다.

# getting full dataset
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)


# First train an SVD algorithm on the movielens dataset.

# 전체 trainset을 빌드하고 평가를 위해 교차 유효성 검사를 사용합니다.
trainset = data.build_full_trainset() # 트레인셋을 만들고
svd.fit(trainset) # train 셋을 통해 svd를 fit 하는 작업 인듯?...
# svd.train(trainset)

user_785314['Estimate_Score'] = user_785314['Movie_Id'].apply(lambda x: svd.predict(785314, x).est) # 785314사용자에 대해 예측하는 작업인듯 예상점수를 나타낸다.

user_785314 = user_785314.drop('Movie_Id', axis = 1) 

user_785314 = user_785314.sort_values('Estimate_Score', ascending=False) # 점수에 따라 정렬
print(user_785314.head(10))

In [ ]:
def recommend(movie_title, min_count):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['Name'] == movie_title][0]) # df_title['Name'] 과  movie_title 이 같은 것을 
    target = df_p[i]  # 피벗테이블에서 i 에 해당하는 애들을 타겟으로 잡는다.
    similar_to_target = df_p.corrwith(target) #피벗테이블과 타겟관의 상관관계를 나타낸다. 이것을 similar_to_target 이라는 변수에 담는다.
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR']) #  데이터 프레임을 만든다.
    corr_target.dropna(inplace = True) # 결측치를 제거한다.
    corr_target = corr_target.sort_values('PearsonR', ascending = False) # PearsonR 을 통한 데이터를 sort 해준다.
    corr_target.index = corr_target.index.map(int) # index를 int형으로 생성해준다.
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']] # 이 4가지 열만 사용한다. join은 알아서 같은 걸로 묶어주나보다.
    print(corr_target[corr_target['count']>min_count][:10].to_string(index=False)) # index는 출력안하고 마지막 10개만 출력하게끔 한다. 위에서 sort해서 가능하다.

In [ ]:
recommend("What the #$*! Do We Know!?", 0)


In [ ]:
recommend("X2: X-Men United", 0)

In [ ]:
print(df_movie_summary)
print(df_title)